ASAP CRN Metadata compilation

# Team Hardy. ASAP CRN Metadata scrubbing

Compare with Team Hardy scRNAseq data and confirm bulk ASSAY

19 Sept 2024
Andy Henrie





In [2]:
import pandas as pd

from pathlib import Path

%load_ext autoreload
%autoreload 2

In [4]:

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hardy/Hardy_bulkrnaseq_pm_hs/**/*.gz" > hardy_hexhash.log



using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



### Helpers (eventually load these from validate.py + io_helpers.py, etc)

In [6]:
def read_CDE(metadata_version:str="v3.0", local_path:str|bool|Path=False):
    """
    Load CDE from local csv and cache it, return a dataframe and dictionary of dtypes
    """
    # Construct the path to CSD.csv
    GOOGLE_SHEET_ID = "1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc"

    if metadata_version == "v1":
        sheet_name = "ASAP_CDE_v1"
    elif metadata_version == "v2":
        sheet_name = "ASAP_CDE_v2"
    elif metadata_version == "v2.1":
        sheet_name = "ASAP_CDE_v2.1"
    elif metadata_version == "v3.0-beta":
        sheet_name = "ASAP_CDE_v3.0-beta"
    elif metadata_version in ["v3.0", "v3.0.0"]:
        sheet_name = "ASAP_CDE_v3"
    else:
        sheet_name = "ASAP_CDE_v3.0"


    if metadata_version in ["v1","v2","v2.1","v3.0-beta"]:
        print(f"metadata_version: {sheet_name}")
    else:
        print(f"Unsupported metadata_version: {sheet_name}")
        return 0,0
    
    cde_url = f"https://docs.google.com/spreadsheets/d/{GOOGLE_SHEET_ID}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    if local_path:
        cde_url = Path(local_path) / f"{sheet_name}.csv"
        print(cde_url)

    
    try:
        CDE_df = pd.read_csv(cde_url)
        read_source = "url" if not local_path else "local file"
        print(f"read {read_source}")
    except:
        CDE_df = pd.read_csv(f"{sheet_name}.csv")
        print("read local file")

    # drop rows with no table name (i.e. ASAP_ids)
    CDE_df.dropna(subset=['Table'], inplace=True)

    return CDE_df


# Function to parse the file to extract MD5 and filenames
def extract_md5_from_details(md5_file):
    md5s = {}
    with open(md5_file, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            if line.startswith("gs://"):
                current_file = line.strip().rstrip(":")
                current_file = current_file.split("/")[-1]
            if "Hash (md5)" in line:
                md5s[current_file] = line.split(":")[1].strip()
    return md5s


# Function to parse the file to extract MD5 and filenames
def extract_md5_from_details2(md5_file):
    md5s = {}
    with open(md5_file, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            if line.startswith("Hashes [hex]"):
                current_file = line.strip().rstrip(":")
                current_file = current_file.split("/")[-1]
            if "Hash (md5)" in line:
                md5s[current_file] = line.split(":")[1].strip()
    return md5s



# Function to parse the file to extract crc32c and filenames
def extract_crc32c_from_details2(md5_file):
    crcs = {}
    with open(md5_file, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            if line.startswith("Hashes [hex]"):
                current_file = line.strip().rstrip(":")
                current_file = current_file.split("/")[-1]
            if "Hash (crc32c)" in line:
                crcs[current_file] = line.split(":")[1].strip()
    return crcs



# Function to parse the file to extract crc32c and filenames
def extract_hashes_from_gcloudstorage(source_hash):

    crcs = {}
    md5s = {}

    with open(source_hash, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            
            if line.startswith("crc32c_hash:"):
                curr_crc =  line.split(":")[1].strip()

            elif line.startswith("md5_hash:"):
                curr_md5 =  line.split(":")[1].strip()

            elif line.startswith("url:"):
                current_file = line.split("/")[-1].strip()
                crcs[current_file] = curr_crc
                md5s[current_file] = curr_md5
            # else:
            #     print(f'cruff:{line.strip()}')


    return crcs, md5s



# Function to parse the file to extract crc32c and filenames
def extract_hashes_from_gsutil(source_hash):

    crcs = {}
    md5s = {}

    with open(source_hash, "r") as f:
        lines = f.readlines()
        current_file = None
        for line in lines:
            if line.startswith("Hashes [hex]"):
                current_file = line.strip().rstrip(":")
                current_file = current_file.split("/")[-1]
            if "Hash (crc32c)" in line:
                crcs[current_file] = line.split(":")[1].strip()
            if "Hash (md5)" in line:
                md5s[current_file] = line.split(":")[1].strip()

    return crcs, md5s

NULL = "NA"

def read_file(data_file):
    """
    TODO: depricate dtypes
    """
    encoding = 'latin1'

    print(f"reading {data_file} txt/csv, encoding={encoding}")
 
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.encode('latin1', errors='replace').str.decode('utf-8', errors='replace')

    df.replace({"":NULL, pd.NA:NULL}, inplace=True)

    return df
    


In [7]:
Path.cwd()

PosixPath('/Users/ergonyc/Projects/ASAP/harmonized-wf-dev/data/teams/hardy')

In [12]:

bucket_files_md5 = extract_md5_from_details2("bulkRNAseq/hardy_hexhash.log")


In [13]:
bucket_files_md5all

{'Hardy_001_R1.fastq.gz': 'c78d76759a539f203eafc53165f14ab2',
 'Hardy_001_R2.fastq.gz': 'c7b906228c37548b2bdf524dfceb6d3d',
 'Hardy_002_R1.fastq.gz': 'fe7312452f2f885ff259d53a398928db',
 'Hardy_002_R2.fastq.gz': '19410a3578dc9f8bb3a60ec20bae1263',
 'Hardy_003_R1.fastq.gz': 'b87c8d1faabb0c0de7296403ae39fe7c',
 'Hardy_003_R2.fastq.gz': '94230099e7b9cca6ceb2ab76f20d4393',
 'Hardy_004_R1.fastq.gz': 'f9f193fac251a640f40afe45931305c3',
 'Hardy_004_R2.fastq.gz': '885b9cd6a202f4817eb34da669de5c97',
 'Hardy_005_R1.fastq.gz': '2fc451d532fa87afa0772296adc3148e',
 'Hardy_005_R2.fastq.gz': '2843457c150147ca557c38c7df80d7d0',
 'Hardy_006_R1.fastq.gz': 'fbe9a27ad129531cde8f4ef1e4a4460e',
 'Hardy_006_R2.fastq.gz': 'c662d4f290b15b7c1644b66bef6237f4',
 'Hardy_007_R1.fastq.gz': '103ef7739335d6aab872b86c0fa09719',
 'Hardy_007_R2.fastq.gz': 'e44a3a574ee3b5f22260833516d1cc8b',
 'Hardy_008_R1.fastq.gz': 'baae40654e4fd99320ab79112628cd7a',
 'Hardy_008_R2.fastq.gz': 'b4bf6d86c50352955fd7913af33b4f36',
 'Hardy_

In [15]:
df = pd.DataFrame(bucket_files_md5.items(), columns=["filename", "md5"])
df


,filename,md5
0,Hardy_001_R1.fastq.gz,c78d76759a539f203eafc53165f14ab2
1,Hardy_001_R2.fastq.gz,c7b906228c37548b2bdf524dfceb6d3d
2,Hardy_002_R1.fastq.gz,fe7312452f2f885ff259d53a398928db
3,Hardy_002_R2.fastq.gz,19410a3578dc9f8bb3a60ec20bae1263
4,Hardy_003_R1.fastq.gz,b87c8d1faabb0c0de7296403ae39fe7c
...,...,...
571,Hardy_286_R2.fastq.gz,3e03c10459cad5577daffc4d23c0cf3f
572,Hardy_287_R1.fastq.gz,b5da48a22997826cad1918656828ebb3
573,Hardy_287_R2.fastq.gz,b914ae8aa9dd697c62caed250790e585
574,Hardy_288_R1.fastq.gz,185a9723cb07a0ef1561e1670a9957fc


## import uploaded Table
write clean metadata tables according to CDE v1

### Team Hardy

In [ ]:
## convert 
metadata_path = Path.home() / ("Projects/ASAP/meta-clean") / "clean/team-Lee/v2"

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv", index_col=0)
SUBJECT = read_file(f"{metadata_path}/SUBJECT.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv", index_col=0)
SAMPLE = read_file(f"{metadata_path}/SAMPLE.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv", index_col=0)
CLINPATH = read_file(f"{metadata_path}/CLINPATH.csv")

# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv", index_col=0)
STUDY = read_file(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv", index_col=0)
PROTOCOL = read_file(f"{metadata_path}/PROTOCOL.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv", index_col=0)
DATA = read_file(f"{metadata_path}/DATA.csv")


reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/SUBJECT.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/SAMPLE.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/CLINPATH.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/STUDY.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/PROTOCOL.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/DATA.csv txt/csv, encoding=latin1


In [ ]:
bool(Path.cwd())

True

In [ ]:
metadata_version = "v3.0-beta"
CDE_df = read_CDE(metadata_version, local_path=Path.cwd())




metadata_version: ASAP_CDE_v3.0.0-beta
/Users/ergonyc/Projects/ASAP/harmonized-wf-dev/data/teams/lee/ASAP_CDE_v3.0.0-beta.csv
read local file


In [ ]:
CDE_df.head()

,Table,Field,Description,DataType,Required,Validation,V0,comment,denormalized,dataset relavent
2,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",NaN,NaN,NaN,NaN
3,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,NaN,NaN,NaN,NaN
4,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,NaN,NaN,NaN,NaN
5,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN,NaN,NaN,NaN,NaN
6,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,NaN,NaN,NaN,NaN


In [ ]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R1_001.fastq.gz,Raw sequencing data,9977258e598d6a52130c29c71aef6925,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
1,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R2_001.fastq.gz,Raw sequencing data,fe2cf93257801227b7072a4fb7d18792,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
2,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R1_001.fastq.gz,Raw sequencing data,110ca4864cf6938faca67567bebfb6cc,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
3,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R2_001.fastq.gz,Raw sequencing data,0dcc67217e43ab53bae0d0676f9bfe8b,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
4,MFG_PD_0009,rep1,1,0,BATCH_4,fastq,MFGPD0009_S3_L001_R1_001.fastq.gz,Raw sequencing data,a2608d0bd192333b0076d7091c1c50ea,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)


In [ ]:
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
1,HIP_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_9,Brain,Hippocampus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
2,SN_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_7,Brain,Substantia_Nigra,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
3,MFG_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
4,HIP_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_9,Brain,Hippocampus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA


In [ ]:
# merge df and DATA on sample_id
DATA_ = df.merge(DATA, left_on="sample_id", right_on="sample_id", how="left")


In [ ]:
# drop duplicates on md5
DATA_.drop_duplicates(subset="md5", inplace=True)

df.shape, DATA.shape, DATA_.shape

((50, 5), (150, 17), (50, 21))

In [ ]:
DATA_.columns

Index(['filename', 'md5', 'id', 'subject_id', 'sample_id', 'replicate',
       'replicate_count', 'repeated_sample', 'batch', 'file_type', 'file_name',
       'file_description', 'file_MD5', 'technology', 'omic', 'adjustment',
       'content', 'time', 'header', 'annotation', 'configuration_file'],
      dtype='object')

copy 'filename' -> 'file_name'

copy  'md5' -> 'file_md5'

change 'technology' -> 'bulk'

In [ ]:
DATA_['file_MD5'] = DATA_['md5']
DATA_['file_name'] = DATA_['filename']
DATA_['technology'] = 'bulk'

# set batch to "batch_x"
DATA_['batch'] = "batch_x"

Next take the unique sample_id and subset SAMPLE, and CLINPATH to only include the bulk samples.


In [ ]:
sample_ids = DATA_['sample_id'].unique()
subject_ids = DATA_['subject_id'].unique()
sample_ids, subject_ids

(array(['MFG_PD_0009', 'MFG_PD_0348', 'MFG_PD_0413', 'MFG_HC_0602',
        'MFG_HC_1225', 'MFG_HC_1308', 'MFG_PD_1312', 'MFG_PD_1317',
        'MFG_PD_1344', 'MFG_PD_1441', 'MFG_PD_1504', 'MFG_PD_1858',
        'MFG_HC_1862', 'MFG_HC_1864', 'MFG_PD_1902', 'MFG_PD_1921',
        'MFG_HC_1939', 'MFG_PD_1973', 'MFG_PD_2005', 'MFG_PD_2038',
        'MFG_HC_2057', 'MFG_PD_2058', 'MFG_HC_2061', 'MFG_HC_2062',
        'MFG_HC_2067'], dtype=object),
 array(['PD_0009', 'PD_0348', 'PD_0413', 'HC_0602', 'HC_1225', 'HC_1308',
        'PD_1312', 'PD_1317', 'PD_1344', 'PD_1441', 'PD_1504', 'PD_1858',
        'HC_1862', 'HC_1864', 'PD_1902', 'PD_1921', 'HC_1939', 'PD_1973',
        'PD_2005', 'PD_2038', 'HC_2057', 'PD_2058', 'HC_2061', 'HC_2062',
        'HC_2067'], dtype=object))

In [ ]:
SAMPLE_ = SAMPLE[SAMPLE['sample_id'].isin(sample_ids)].copy()
CLINPATH_ = CLINPATH[CLINPATH['subject_id'].isin(subject_ids)].copy()


In [ ]:

SUBJECT_ = SUBJECT[SUBJECT['subject_id'].isin(subject_ids)].copy()

In [ ]:
# reindex SAMPLE_ and CLINPATH_
SAMPLE_.reset_index(inplace=True, drop=True)
CLINPATH_.reset_index(inplace=True, drop=True)


# set batch to batch_x
SAMPLE_['batch'] = "batch_x"

In [ ]:
SAMPLE_

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
1,MFG_HC_0602,HC_0602,06-02,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
2,MFG_PD_0009,PD_0009,00-09,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
3,MFG_PD_1921,PD_1921,19-21,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
4,MFG_PD_2058,PD_2058,20-58,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
5,MFG_PD_1441,PD_1441,14-41,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
6,MFG_PD_1344,PD_1344,13-44,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
7,MFG_HC_1939,HC_1939,19-39,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
8,MFG_HC_1308,HC_1308,13-08,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
9,MFG_HC_1862,HC_1862,18-62,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA


In [ ]:
# wrape this in try/except to make suing the ReportCollector portable
# probably an abstract base class would be better

class DummyStreamlit:
    @staticmethod
    def markdown(self,msg):
        pass
    def error(self,msg):
        pass
    def header(self,msg):
        pass        
    def subheader(self,msg):
        pass    
    def divider(self):
        pass
st = DummyStreamlit()


def get_log(log_file):
    """ grab logged information from the log file."""
    with open(log_file, 'r') as f:
        report_content = f.read()
    return report_content

def columnize( itemlist ):
    NEWLINE_DASH = ' \n- '
    if len(itemlist) > 1:
        return f"- {itemlist[0]}{NEWLINE_DASH.join(itemlist[1:])}"
    else:
        return f"- {itemlist[0]}"
    
def read_meta_table(table_path):
    # read the whole table
    try:
        table_df = pd.read_csv(table_path,dtype=str)
    except UnicodeDecodeError:
        table_df = pd.read_csv(table_path, encoding='latin1',dtype=str)

    # drop the first column if it is just the index
    if table_df.columns[0] == "Unnamed: 0":
        table_df = table_df.drop(columns=["Unnamed: 0"])

    return table_df


class ReportCollector:
    """
    Class to collect and log messages, errors, and markdown to a log file and/or streamlit
    """

    def __init__(self, destination="both"):
        self.entries = []
        self.filename = None

        if destination in ["both", "streamlit"]:
            self.publish_to_streamlit = True
        else:
            self.publish_to_streamlit = False


    def add_markdown(self, msg):
        self.entries.append(("markdown", msg))
        if self.publish_to_streamlit:
            st.markdown(msg)


    def add_error(self, msg):
        self.entries.append(("error", msg))
        if self.publish_to_streamlit:
            st.error(msg)

    def add_header(self, msg):
        self.entries.append(("header", msg))
        if self.publish_to_streamlit:    
            st.header(msg)

    def add_subheader(self, msg):
        self.entries.append(("subheader", msg))
        if self.publish_to_streamlit:    
            st.subheader(msg)

    def add_divider(self):
        self.entries.append(("divider", None))
        if self.publish_to_streamlit:    
            st.divider()

    
    def write_to_file(self, filename):
        self.filename = filename
        with open(filename, 'w') as f:
            report_content = self.get_log()
            f.write(report_content)
    

    def get_log(self):
        """ grab logged information from the log file."""
        report_content = []
        for msg_type, msg in self.entries:
            if msg_type == "markdown":
                report_content += msg + '\n'
            elif msg_type == "error":
                report_content += f"🚨⚠️❗ **{msg}**\n"
            elif msg_type == "header":
                report_content += f"# {msg}\n"
            elif msg_type == "subheader":
                report_content += f"## {msg}\n"
            elif msg_type == "divider":
                report_content += 60*'-' + '\n'
        
        return "".join(report_content)

    def reset(self):
        self.entries = []
        self.filename = None

    def print_log(self):
        print(self.get_log())


def validate_table(df: pd.DataFrame, table_name: str, specific_cde_df: pd.DataFrame, out: ReportCollector ):
    """
    Validate the table against the specific table entries from the CDE
    """
    df.replace({"":NULL, pd.NA:NULL}, inplace=True)
    def my_str(x):
        return f"'{str(x)}'"
    missing_required = []
    missing_optional = []
    null_fields = []
    invalid_entries = []
    total_rows = df.shape[0]
    df_out = pd.DataFrame()
    for field in specific_cde_df["Field"]:
        entry_idx = specific_cde_df["Field"]==field
        print(f"validating {field}")
        
        opt_req = "REQUIRED" if specific_cde_df.loc[entry_idx, "Required"].item()=="Required" else "OPTIONAL"

        if field not in df.columns:
            if opt_req == "REQUIRED":
                missing_required.append(field)
            else:
                missing_optional.append(field)

            # print(f"missing {opt_req} column {field}")
            df_out[field] = [NULL]*total_rows

        else:
            datatype = specific_cde_df.loc[entry_idx,"DataType"]
            if datatype.item() == "Integer":
                # recode "Unknown" as NULL
                df.replace({"Unknown":NULL, "unknown":NULL}, inplace=True)
                df[field].apply(lambda x: int(x) if x!=NULL else x )
                # test that all are integer or NULL, flag NULL entries
            elif datatype.item() == "Float":
                # recode "Unknown" as NULL
                df.replace({"Unknown":NULL, "unknown":NULL}, inplace=True)
                df[field].apply(lambda x: float(x) if x!=NULL else x )
                # test that all are float or NULL, flag NULL entries
            elif datatype.item() == "Enum":

                valid_values = eval(specific_cde_df.loc[entry_idx,"Validation"].item())
                entries = df[field]
                valid_entries = entries.apply(lambda x: x in valid_values)
                invalid_values = entries[~valid_entries].unique()
                n_invalid = invalid_values.shape[0]
                if n_invalid > 0:
                    valstr = ', '.join(map(my_str, valid_values))
                    invalstr = ', '.join(map(my_str,invalid_values))
                    invalid_entries.append((opt_req, field, n_invalid, valstr, invalstr))
            else: #dtype == String
                pass
            
            n_null = (df[field]==NULL).sum()
            if n_null > 0:            
                null_fields.append((opt_req, field, n_null))
            
            df_out[field] = df[field]
            

    # now compose report...
    if len(missing_required) > 0:
        out.add_error(f"Missing Required Fields in {table_name}: {', '.join(missing_required)}")
    else:
        out.add_markdown(f"All required fields are present in *{table_name}* table.")

    if len(missing_optional) > 0:
        out.add_error(f"Missing Optional Fields in {table_name}: {', '.join(missing_optional)}")
    

    if len(null_fields) > 0:
        # print(f"{opt_req} {field} has {n_null}/{df.shape[0]} NULL entries ")
        out.add_error(f"{len(null_fields)} Fields with empty (NULL) values:")
        for opt_req, field, count in null_fields:
            out.add_markdown(f"\n\t- {field}: {count}/{total_rows} empty rows ({opt_req})")
    else:
        out.add_markdown(f"No empty entries (NULL) found .")


    if len(invalid_entries) > 0:
        out.add_error(f"{len(invalid_entries)} Fields with invalid entries:")
        for opt_req, field, count, valstr, invalstr in invalid_entries:
            str_out = f"- _*{field}*_:  invalid values 💩{invalstr}\n"
            str_out += f"    - valid ➡️ {valstr}"
            out.add_markdown(str_out)
    else:
        out.add_markdown(f"No invalid entries found in Enum fields.")


    return df, df_out, out


report = ReportCollector(destination="log")

df = DATA_.copy()
table_choice = "DATA"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

DATA_export, DATA_out, report = retval

validating sample_id
validating replicate
validating replicate_count
validating repeated_sample
validating batch
validating file_type
validating file_name
validating file_description
validating file_MD5
validating adjustment
validating content
validating header
validating annotation
validating configuration_file
All required fields are present in *DATA* table.
🚨⚠️❗ **2 Fields with empty (NULL) values:**

	- header: 50/50 empty rows (OPTIONAL)

	- annotation: 50/50 empty rows (OPTIONAL)
No invalid entries found in Enum fields.



In [ ]:
report = ReportCollector(destination="log")

df = SAMPLE_.copy()
table_choice = "SAMPLE"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

validating sample_id
validating subject_id
validating source_sample_id
validating replicate
validating replicate_count
validating repeated_sample
validating batch
validating condition
validating tissue
validating time
validating alternate_id
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Missing Optional Fields in SAMPLE: condition, time, alternate_id**
No empty entries (NULL) found .
No invalid entries found in Enum fields.



In [ ]:
SAMPLE_export, SAMPLE_out, report = retval

In [ ]:
SAMPLE_export.shape, SAMPLE_.shape, SAMPLE.shape, SAMPLE_out.shape

((25, 33), (25, 33), (75, 33), (25, 11))

In [ ]:
report = ReportCollector(destination="log")

df = CLINPATH_.copy()
table_choice = "CLINPATH"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

CLINPATH_export, CLINPATH_out, report = retval



validating subject_id
validating source_subject_id
validating duration_pmi
validating age_at_death
validating path_autopsy_dx_main
validating path_autopsy_second_dx
validating path_autopsy_third_dx
validating path_autopsy_fourth_dx
validating path_autopsy_fifth_dx
validating path_autopsy_sixth_dx
validating path_autopsy_seventh_dx
validating path_autopsy_eight_dx
validating path_year_death
validating cause_death
validating other_cause_death_1
validating other_cause_death_2
validating brain_weight
validating path_braak_nft
validating path_braak_asyn
validating path_cerad
validating path_thal
validating known_pathogenic_mutation
validating PD_pathogenic_mutation
validating path_mckeith
validating sn_neuronal_loss
validating path_infarcs
validating path_nia_ri
validating path_nia_aa_a
validating path_nia_aa_b
validating path_nia_aa_c
validating TDP43
validating arteriolosclerosis_severity_scale
validating amyloid_angiopathy_severity_scale
validating path_ad_level
validating dig_slide_avai

In [ ]:
import numpy as np
df.replace({"":NULL, pd.NA:NULL, np.NaN:NULL}, inplace=True)


## Clean V1 Table
write clean metadata tables according to CDE v1

### Team Hardy

In [3]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-hardy")
metadata_path = data_path / "metadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [4]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [5]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [6]:

STUDY = prep_table(STUDY, CDE_df)
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE


In [7]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [8]:
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,Mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,Aine.fairbrother-browne.18@ucl.ac.uk,...,Nan,Nan,Nan,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,Https://scholar.google.co.uk/citations?user=lt...,Nan,v1_20240516


In [9]:
# fix STUDY formatting
tmp = pd.DataFrame()
tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
PROTOCOL = tmp.drop([0])
PROTOCOL.head()

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
1,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NaN


In [10]:
# fix the column order
PROTOCOL = prep_table(PROTOCOL, CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()



All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [11]:

SUBJECT = prep_table(SUBJECT, CDE_df)

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in SUBJECT. 🥳



In [12]:
SUBJECT.race.unique()[0]

'Nan'

In [13]:
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,Nan,Nan,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,Nan,Nan,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,Nan,Nan,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,Nan,Nan,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,Nan,Nan,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [14]:
# def clean_empty_cols(df):
#     for col in df.columns:
#         if df[col].dtype in ["object", "string"]:
#             df[col].replace("Nan", pd.NA, inplace=True)
#             df[col].replace("nan", pd.NA, inplace=True)
#             df[col].replace("NA", pd.NA, inplace=True)

#         elif df[col].dtype in ["int", "float"]:
#             df[col].fillna(pd.NA, inplace=True)
#     return df


# SUBJECT = clean_empty_cols(SUBJECT)

SUBJECT.replace("Nan", pd.NA, inplace=True)
# SUBJECT.replace("nan", "", inplace=True)
# SUBJECT.fillna("", inplace=True)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,<NA>,<NA>,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,<NA>,<NA>,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,<NA>,<NA>,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,<NA>,<NA>,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,<NA>,<NA>,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [15]:
SAMPLE[['assay','sequencing_end', 'sequencing_length', 'sequencing_instrument']]

,assay,sequencing_end,sequencing_length,sequencing_instrument
0,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
1,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
2,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
4,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
...,...,...,...,...
3611,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3612,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3613,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3614,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000


In [47]:
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

SAMPLE = prep_table(SAMPLE, CDE_df)

# allow sequence_length == 190 for now

# SAMPLE.fillna("", inplace=True)
# remove any text nans
SAMPLE.replace("Nan", pd.NA, inplace=True)
SAMPLE.replace("nan", pd.NA, inplace=True)
SAMPLE.replace("NA", pd.NA, inplace=True)
# SAMPLE = clean_empty_cols(SUBJECT)
# SAMPLE.fillna("", inplace=True)

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()




# add 'replicate' coding (nans)

🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'190'
    - valid ➡️ '25', '50', '100', '150'



In [48]:

SAMPLE['source_sample_id'] = pd.NA
SAMPLE['DV200'] = pd.NA
# SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)


In [49]:
SAMPLE['replicate'].replace({pd.NA: "Rep1", "rep1": "Rep1", "rep2": "Rep2"}, inplace=True)

In [50]:
SAMPLE.head()

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,source_sample_id,DV200
0,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
1,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
2,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
3,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
4,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>


In [51]:
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")

CLINPATH = prep_table(CLINPATH, CDE_df)


# CLINPATH.replace("Nan", pd.NA, inplace=True)
CLINPATH.replace("nan", "Nan", inplace=True)
CLINPATH.replace("NA", "Nan", inplace=True)

# CLINPATH.fillna("Nan")
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




🚨⚠️❗ **Missing Required Fields in CLINPATH: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_autopsy_dx_main*_:  invalid values 💩'Control brain', 'Pathological ageing', 'Control brain / Path ageing', 'Argyrophilic grain disease', 'Control brain, Cerebrovascular disease (small vessel)', 'Cerebrovascular disease (small vessel)', 'Control brain, Alzheimer`s disease (intermediate level AD pathological change)', 'Control brain / Path ageing, CAA'
    - valid ➡️ 'Lewy body disease nos', 'Parkinson's disease', 'Parkinson's disease with dementia', 'Dementia with Lewy bodies', 'Multiple system atrophy (SND>OPCA)', 'Multiple system atrophy (OPCA<SND)', 

In [52]:
na_float2intstr = lambda val: str(int(float(val))) if val != 'Nan' else pd.NA 

# replace 'path_braak_asyn' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_asyn'] = CLINPATH['path_braak_asyn'].apply(na_float2intstr)



In [53]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].apply(na_float2intstr).replace({"0":"0", 
                                                                                        "1":"I", 
                                                                                        "2": "II", 
                                                                                        "3":"III", 
                                                                                        "4":"IV", 
                                                                                        "5":"V", 
                                                                                        "6":"VI"})


In [54]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"



# check APOE_e4_status ? currently empty

# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 

CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})


In [55]:

CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].apply(na_float2intstr).replace({"0":"A0", 
                                                                                    "1":"A1", 
                                                                                    "2": "A2", 
                                                                                    "3":"A3"})


In [56]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].apply(na_float2intstr).replace({"0":"B0", 
                                                                                    "1":"B1", 
                                                                                    "2": "B2", 
                                                                                    "3":"B3"})


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].apply(na_float2intstr).replace({"0":"C0", 
                                                                                    "1":"C1", 
                                                                                    "2": "C2", 
                                                                                    "3":"C3"})



In [57]:



CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})





In [58]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [59]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)

In [60]:
CLINPATH['path_autopsy_dx_main'].unique()

array(["Parkinson's disease with dementia", "Parkinson's disease",
       'Control, no misfolded protein or significant vascular pathology',
       'Control, Argyrophilic grain disease',
       'Control, Cerebrovascular disease (atherosclerosis)',
       "Alzheimer's disease (intermediate level neuropathological change)",
       'Control, Cerebrovascular disease (cerebral amyloid angiopathy)',
       'Nan'], dtype=object)

In [61]:
# remove any text nans
CLINPATH.replace("Nan", pd.NA, inplace=True)



In [62]:
CLINPATH['source_sample_id'] = pd.NA

# CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)


In [63]:

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*smoking_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Current smoker', 'Former smoker', 'Never', 'Unknown'
- _*APOE_e4_status*_:  invalid values 💩'<NA>'
    - valid ➡️ '22', '23

In [64]:


# # write the clean metadata
# STUDY.to_csv(data_path / "metadata/STUDY.csv")
# PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
# CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
# SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
# SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# # also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hardy"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")



## Update the table to v2

In [65]:
from update_schema import update_tables_to_CDEv2

tables_path = Path.cwd() / "clean/team-Hardy"


CDEv1 = pd.read_csv( Path.cwd() / "ASAP_CDE_v1.csv" )
CDEv2 = pd.read_csv( Path.cwd() / "ASAP_CDE_v2.csv" )


STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = update_tables_to_CDEv2(tables_path, CDEv1, CDEv2)


In [66]:

export_root = Path.cwd() / "clean/team-Hardy/v2"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


In [67]:

STUDYv2.to_csv( export_root / "STUDY.csv")
PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv")
SAMPLEv2.to_csv(export_root / "SAMPLE.csv")
SUBJECTv2.to_csv(export_root / "SUBJECT.csv")
CLINPATHv2.to_csv(export_root / "CLINPATH.csv")
DATAv2.to_csv(export_root / "DATA.csv")


Transfer cleaned metadata to raw buckets 




## transfer metadata Hardy 

In [68]:
# Hardy
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json 


Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [69]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-hardy/"


        20  2023-08-15T15:13:44Z  gs://asap-raw-data-team-hardy/test-raw-hardy.txt#1692112424343862  metageneration=1
        19  2023-08-17T09:58:09Z  gs://asap-raw-data-team-hardy/testfile.txt#1692266289818249  metageneration=1
                                 gs://asap-raw-data-team-hardy/fastqs/
                                 gs://asap-raw-data-team-hardy/metadata/
                                 gs://asap-raw-data-team-hardy/ogmetadata/
TOTAL: 2 objects, 39 bytes (39 B)


## fix bucket file structure to match the prescribed convention

change "fastq" -> "fastqs"

In [4]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-hardy/fastq" "gs://asap-raw-data-team-hardy/fastqs"


Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R2_001.fastq.gz [Content-Type=application/

In [6]:
!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-hardy/fastqs" 

gs://asap-raw-data-team-hardy/fastqs/all_fastq.txt
gs://asap-raw-data-team-hardy/fastqs/220520_A01366_0202_AHL5JVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0009_AHMYFTDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0010_BHMYM5DSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0013_BHLNTGDSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0014_AHN3MVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0015_BH3NFNDSX5/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0016_AHF7KJDMXY/
gs://asap-raw-data-team-hardy/fastqs/221013_A01897_0034_AHGFTNDMXY/


In [70]:

# !gsutil -u dnastack-asap-parkinsons cp -r ./clean/team-Hardy/v2_20231128  "gs://asap-raw-data-team-hardy/metadata/v2"
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Hardy/v2_20231201/*.csv"  "gs://asap-raw-data-team-hardy/metadata/v2"


Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [72]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [73]:
!gsutil  cp -r "./clean/team-Hardy/v2_20231201/*.csv" "gs://asap-workflow-dev/metadata/v2/hardy"

Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

## check file md5s

In [35]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [35]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json  



Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [36]:

# !gcloud storage hash "gs://asap-raw-data-team-hardy/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hardy/**/*.gz" > hardy_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [37]:
bucket_files_md5 = extract_md5_from_details2("hardy_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)




/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [1]:
! pip freeze

anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1693488585952/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
argcomplete==1.12.3
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1695386808436/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1683424013410/work
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1698174530262/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backport

In [38]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [46]:
checksum.head()

,file_name,file_MD5,check
0,babom_ACG_S20_L008_R1_001.fastq.gz,F37738d04879c1cb8c380920964dad4d,f37738d04879c1cb8c380920964dad4d
1,babom_ACG_S20_L008_I1_001.fastq.gz,F3053c59a33ce48cbea6a602dcc491c7,f3053c59a33ce48cbea6a602dcc491c7
2,babom_ACG_S20_L008_R2_001.fastq.gz,69fe97e968dc1cb782a3d995918b5bc8,69fe97e968dc1cb782a3d995918b5bc8
3,babom_ACG_S20_L008_I2_001.fastq.gz,4eeee300289a4fd18169dddbbfd8ad72,4eeee300289a4fd18169dddbbfd8ad72
4,babom_ACG_S20_L007_R1_001.fastq.gz,7b321fc0e55ffbd496a241dcb0746def,7b321fc0e55ffbd496a241dcb0746def


In [ ]:
# these are actually matches... probably unstripped line endings.